# Analyze database
This notebook is used to analyze the database ant its recordings.

In [1]:
import sqlite3
from pathlib import Path
from typing import TypedDict

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

from ddlitlab2024 import DB_PATH

In [2]:
DB_PATH = Path("/srv/ssd_nvm/dataset/ddlitlab2024/db/robocup_2024_german_open_2025.sqlite3")  # TODO Remove me

In [3]:
print(f"Analyzing {DB_PATH}")

# Connect read-only to the SQLite database
conn = sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True)

Analyzing /srv/ssd_nvm/dataset/ddlitlab2024/db/robocup_2024_german_open_2025.sqlite3


## Gather metrics:
- Duration [s]
- Image samples
- Rotation samples
- Joint state samples
- Joint command samples
- Game state samples
- Total number of (previous) samples

In [4]:
class Metric(TypedDict):
    name: str
    duration_s: float
    num_images: int
    num_rotations: int
    num_joint_states: int
    num_joint_commands: int
    num_game_states: int
    num_messages: int

In [5]:
# Iterate over all recordings in the database and gather metrics
def gather_metrics() -> pd.DataFrame:
    # Get all recordings
    recordings = conn.execute("SELECT _id, original_file, start_time, end_time FROM Recording").fetchall()
    metrics = []

    for recording in recordings:
        recording_id = recording[0]

        duration_s = (datetime.fromisoformat(recording[3]) - datetime.fromisoformat(recording[2])).total_seconds()

        # Get the number of images
        num_images = conn.execute(
            "SELECT COUNT(*) FROM Image WHERE recording_id = ?", (recording_id,)
        ).fetchone()[0]

        # Get the number of IMU messages
        num_rotations = conn.execute(
            "SELECT COUNT(*) FROM Rotation WHERE recording_id = ?", (recording_id,)
        ).fetchone()[0]

        # Get the number of joint states
        num_joint_states = conn.execute(
            "SELECT COUNT(*) FROM JointStates WHERE recording_id = ?", (recording_id,)
        ).fetchone()[0]

        # Get the number of joint commands
        num_joint_commands = conn.execute(
            "SELECT COUNT(*) FROM JointCommands WHERE recording_id = ?", (recording_id,)
        ).fetchone()[0]

        # Get the number of game states
        num_game_states = conn.execute(
            "SELECT COUNT(*) FROM GameState WHERE recording_id = ?", (recording_id,)
        ).fetchone()[0]

        # Get the number of messages
        num_messages = num_images + num_rotations + num_joint_states + num_joint_commands + num_game_states
        metrics.append(
            Metric(
                name=recording[1],
                duration_s=duration_s,
                num_images=num_images,
                num_rotations=num_rotations,
                num_joint_states=num_joint_states,
                num_joint_commands=num_joint_commands,
                num_game_states=num_game_states,
                num_messages=num_messages,
            )
        )

    return pd.DataFrame(metrics)

df = gather_metrics()

In [6]:
df.to_csv("metrics_db.csv")

In [15]:
df["duration_m"] = df["duration_s"] / 60

In [16]:
df = df.sort_values(by="duration_s", ascending=False)
df

,name,duration_s,num_images,num_rotations,num_joint_states,num_joint_commands,num_game_states,num_messages,duration_m
80,ID_donna_2025-03-16T13:50:58_0.mcap,4598.725709,45954,229777,229777,229777,6280,741565,76.645428
59,ID_rory_2025-03-16T14:02:56_0.mcap,3791.524157,37894,189473,189473,189473,5053,611366,63.192069
27,ID_rory_2024-07-19T17:45:15_0.mcap,1903.966492,18877,94395,94395,94395,3273,305335,31.732775
35,ID_donna_2024-07-20T15:26:55_0.mcap,1328.308035,8127,65443,65443,65443,2466,206922,22.138467
82,ID_rory_2025-03-13T10:53:00_0.mcap,1290.537316,12608,63045,63045,63045,2271,204014,21.508955
...,...,...,...,...,...,...,...,...,...
10,ID_jack_2024-07-18T18:29:35_0.mcap,54.169400,369,1852,1852,1852,66,5991,0.902823
57,ID_jack_2024-07-20T15:47:38_0.mcap,47.663031,332,1676,1676,1676,8,5368,0.794384
12,ID_jack_2024-07-18T18:42:01_0.mcap,43.044205,309,1550,1550,1550,53,5012,0.717403
20,ID_rory_2024-07-18T13:52:36_0.mcap,39.836062,237,1191,1191,1191,27,3837,0.663934


In [18]:
# Print latex for each column
for column in df.columns:
    if column == "name":
        continue
    if column in ["duration_s", "duration_m"]:
        print(f"{column} & {df[column].min():.2f} & {df[column].mean():.2f}±{df[column].std():.2f} & {df[column].max():.2f} & {df[column].sum():.2f}\\\\")
    else:
        print(f"{column} & {df[column].min():.0f} & {df[column].mean():.0f}±{df[column].std():.0f} & {df[column].max():.0f} & {df[column].sum():.0f}\\\\")

duration_s & 35.15 & 619.25±665.74 & 4598.73 & 54493.99\\
num_images & 200 & 5952±6638 & 45954 & 523819\\
num_rotations & 1014 & 30147±33378 & 229777 & 2652975\\
num_joint_states & 1014 & 30147±33378 & 229777 & 2652975\\
num_joint_commands & 1014 & 30147±33378 & 229777 & 2652975\\
num_game_states & 1 & 939±1001 & 6280 & 82638\\
num_messages & 3273 & 97334±107712 & 741565 & 8565382\\
duration_m & 0.59 & 10.32±11.10 & 76.65 & 908.23\\
